In [174]:
import requests # type: ignore
import json
import csv
import pandas as pd
import numpy as np

In [175]:
# Load and read json file
with open("request_data.json", "r") as file:
    request_data = json.load(file)

wanted_list = request_data["url"]
request_headers = request_data["headers"]

In [176]:
# Get session
session = requests.Session()
response = session.get(url=wanted_list, headers=request_headers)
data = json.loads(response.text)
session.close()

In [177]:
# Extract base, category, other informations
fbi_data = []

for item in data.get("items", []):
    person = {
        "name": item.get("title"),
        "age_max": item.get("age_max"),
        "age_min": item.get("age_min"),
        "sex": item.get("sex"),
        "place_of_birth": item.get("place_of_birth"),
        "details": item.get("details"),
        "occupations": item.get("occupations") or [],
        "locations": item.get("locations") or [],
        "subjects": item.get("subjects") or [],
        "aliases": item.get("aliases") or []
    }
    fbi_data.append(person)


for person in fbi_data:
    print(f"Name: {person['name']}")
    print(f"Age Max: {person['age_max']}")
    print(f"Age Min: {person['age_min']}")
    print(f"Sex: {person['sex']}")
    print(f"Place of Birth: {person['place_of_birth']}")
    print(f"Details: {person['details']}")
    
    print("Occupations:")
    for occupation in person['occupations']:
        print(f" - {occupation}")

    print("Locations:")
    for location in person['locations']:
        print(f" - {location}")
    
    print("Subjects:")
    for subject in person['subjects']:
        print(f" - {subject}")
    
    print("Aliases:")
    for alias in person['aliases']:
        print(f" - {alias}")
    
    print("\n" + "="*40 + "\n")

In [ ]:
# Transform data
df = pd.DataFrame(fbi_data)

df['details'] = df['details'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
df['occupations'] = df['occupations'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
df['locations'] = df['locations'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
df['subjects'] = df['subjects'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
df['aliases'] = df['aliases'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

df["details"] = df["details"].str.replace("<p>", "").str.replace("</p>", "")
df["age_max"] = pd.to_numeric(df["age_max"], errors='coerce').astype('Int64')
df["age_min"] = pd.to_numeric(df["age_min"], errors='coerce').astype('Int64')

In [ ]:
# Load data to csv
df.to_csv('fbi_data.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

Dados salvos em 'people_data.csv'
